In [ ]:
import pandas as pd
import hashlib
import gc 

In [ ]:
df_train = pd.read_csv(data_folder + 'train.csv').fillna("")
df_test = pd.read_csv(data_folder + 'test.csv').fillna("")

In [ ]:
# Generating a graph of Questions and their neighbors
def generate_qid_graph_table(row):
    hash_key1 = hashlib.md5(row["question1"].encode('utf-8')).hexdigest()
    hash_key2 = hashlib.md5(row["question2"].encode('utf-8')).hexdigest()

    qid_graph.setdefault(hash_key1, []).append(hash_key2)
    qid_graph.setdefault(hash_key2, []).append(hash_key1)

In [ ]:
qid_graph = {}
print('Apply to train...')
df_train.apply(generate_qid_graph_table, axis=1)
print('Apply to test...')
df_test.apply(generate_qid_graph_table, axis=1)

In [ ]:
def pagerank():
    MAX_ITER = 20
    d = 0.85

    # Initializing -- every node gets a uniform value!
    pagerank_dict = {i: 1 / len(qid_graph) for i in qid_graph}
    num_nodes = len(pagerank_dict)

    for iter in range(0, MAX_ITER):

        for node in qid_graph:
            local_pr = 0

            for neighbor in qid_graph[node]:
                local_pr += pagerank_dict[neighbor] / len(qid_graph[neighbor])

            pagerank_dict[node] = (1 - d) / num_nodes + d * local_pr

    return pagerank_dict

In [ ]:
print('Main PR generator...')
pagerank_dict = pagerank()

In [ ]:
def get_pagerank_value(row):
    q1 = hashlib.md5(row["question1"].encode('utf-8')).hexdigest()
    q2 = hashlib.md5(row["question2"].encode('utf-8')).hexdigest()
    s = pd.Series({
        "q1_pr": pagerank_dict[q1],
        "q2_pr": pagerank_dict[q2]
    })
    return s

In [ ]:
pagerank_feats_train = df_train.apply(get_pagerank_value, axis=1)

In [ ]:
del df_train
gc.collect()

In [ ]:
pagerank_feats_test = df_test.apply(get_pagerank_value, axis=1)

## Save raw PageRank

In [ ]:
save_feature_names(['pagerank_q1', 'pagerank_q2'], 'pagerank')

In [ ]:
save_feature_list(pagerank_feats_train, 'train', 'pagerank')

In [ ]:
save_feature_list(pagerank_feats_test, 'test', 'pagerank')